In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.ensemble
import sklearn.tree
import xgboost as xgb
import shap
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap

In [ ]:
def MAPE(true, pred):
    diff = np.abs(np.array(true) - np.array(pred))
    return np.mean(diff / abs(true))
df_gro = pd.read_csv('xgboost95.csv')

columns_to_standardize = df_gro.columns.difference(['site', 'Growth_loss','AI_Class'])
scaler = StandardScaler()
df_gro[columns_to_standardize] = scaler.fit_transform(df_gro[columns_to_standardize])
grouped = df_gro.groupby('AI_Class')
group_humid = grouped.get_group('Humid')

In [ ]:
group_arid = grouped.get_group('Arid')
group_arid = group_arid.drop(['site','AI_Class','Temp','Tmax','Pet','Tmin'],axis=1)
spearman_corr = group_arid.corr(method='spearman')
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data
input_features = group_arid.drop(columns=['Growth_loss'])
vif_df = calculate_vif(input_features)
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 16
plt.figure(figsize=(10,8))
sns.heatmap(round(spearman_corr,2), annot=True, cmap='RdBu')
plt.title('Arid Region')

plt.savefig('FigS9A.jpg', dpi=800, bbox_inches = 'tight')
plt.show()
vif_df.to_csv('Arid_VIF.csv',index=False)

In [ ]:
shap.initjs()
X = group_arid.loc[:, group_arid.columns != 'Growth_loss']
y = group_arid['Growth_loss']
model = xgb.XGBRegressor(max_depth=4, learning_rate=0.07, n_estimators=100, min_child_weight=7, subsample=0.8, colsample_bytree=0.7, reg_lambda=0.4, random_state=30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
model.fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs = -1)
print(np.sqrt(-scores.mean()))
print("Training R2:", r2_score(y_train, Z_1), "RMSE:", np.sqrt(mean_squared_error(y_train, Z_1)), "MAE:", mean_absolute_error(y_train, Z_1), "MAPE:", MAPE(y_train, Z_1)*100,"%")
print("Testing R2:", r2_score(y_test, Z_2), "RMSE:", np.sqrt(mean_squared_error(y_test, Z_2)), "MAE:", mean_absolute_error(y_test, Z_2), "MAPE:", MAPE(y_test,Z_2)*100,"%")
Z_3 = model.predict(X)

xx = np.zeros(shape=(len(X),2))
xx[:,0]=y
xx[:,1]=Z_3
np.savetxt('XGB.csv', xx, delimiter = ',') 
# plot the shear strength prediction results
xx = np.linspace(-0.5, 0.5)
yy = xx
plt.style.use('default')
plt.figure(figsize=(5,4.5),dpi=100)
plt.plot(xx, yy, linewidth=2,c='red')
plt.scatter(y_train, Z_1, marker='o')
plt.scatter(y_test, Z_2, marker='s')

plt.tick_params (axis='both',which='major',labelsize=18)
plt.yticks(fontproperties = 'Times New Roman', size = 18)
plt.xticks(fontproperties = 'Times New Roman', size = 18)

font1 = {'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 20,}
font2 = {'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 15,}
plt.axis('tight')
plt.xlabel('Tested Growth loss', font1)
plt.xticks([-0.5,0,0.5])
plt.yticks([-0.5,0,0.5])
plt.ylabel('Predicted Growth loss', font1)
plt.text(0,0.4,'Arid region', font2)
plt.text(0.1,-0.4,'Training '+"R\u00B2" +'= 0.78', font2)
plt.text(0.1,-0.3,'Testing '+ "R\u00B2"+'= 0.58', font2)
plt.legend(['y = x','Training','Testing'], loc = 'upper left', prop={'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 14,})
plt.savefig('FigS10a.jpg', dpi=600, bbox_inches = 'tight')
plt.show()
X_shap = pd.DataFrame(X, columns = X.columns)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_shap)
print(shap_values.shape)
np.savetxt('shap values.csv', shap_values, delimiter = ',') 
# visualize the j-th prediction's explanation and then the entire set
y_base = explainer.expected_value
group_arid['pred'] = model.predict(X)

shap.force_plot(explainer.expected_value, shap_values[2], X.iloc[2], matplotlib=True)
shap.force_plot(explainer.expected_value, shap_values[16], X.iloc[16], matplotlib=True)
shap.force_plot(explainer.expected_value, shap_values, X)

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)
percentages = mean_abs_shap_values / np.sum(mean_abs_shap_values)
data = pd.DataFrame({
    'feature': X_shap.columns,
    'shap_value': mean_abs_shap_values,
    'percentage': percentages
})

data = data.sort_values(by='shap_value', ascending=True)
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 10))
plt.subplots_adjust(wspace=0.001)
ax1.barh(data['feature'], data['shap_value'], color='lightblue')
for i, (feature, shap_value, percentage) in enumerate(data.itertuples(index=False)):
    ax1.text(shap_value + 0.001, i, f"{shap_value:.3f}", va='center', color='blue')
    ax1.text(shap_value + 0.005, i, f"({percentage:.2%})", va='center', color='red')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_xlabel('Mean(SHAP value)',fontsize=14)
shap.summary_plot(shap_values, X_shap, show=False)
plt.savefig('Fig6a.jpg', dpi=800, bbox_inches = 'tight')
plt.show()

In [ ]:
group_humid = grouped.get_group('Humid')
group_humid = group_humid.drop(['site','AI_Class','Tmax','Tmin','Pet','Temp'],axis=1)
spearman_corr = group_humid.corr(method='spearman')
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data
input_features = group_humid.drop(columns=['Growth_loss'])
vif_df = calculate_vif(input_features)
vif_df.to_csv('Huimd_VIF.csv',index=False)
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 16
plt.figure(figsize=(10,8))
sns.heatmap(round(spearman_corr,2), annot=True, cmap='RdBu')
plt.title('Humid Region')
plt.savefig('FigS9b.jpg', dpi=800, bbox_inches = 'tight')
plt.show()
shap.initjs()
model = xgb.XGBRegressor(max_depth=4, learning_rate=0.07, n_estimators=100, min_child_weight=3, subsample=0.8, colsample_bytree=0.6, reg_lambda=1, random_state=30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
model.fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs = -1)
print(np.sqrt(-scores.mean()))
Z_1 = model.predict(X_train)
Z_2 = model.predict(X_test)
print("Training R2:", r2_score(y_train, Z_1), "RMSE:", np.sqrt(mean_squared_error(y_train, Z_1)), "MAE:", mean_absolute_error(y_train, Z_1), "MAPE:", MAPE(y_train, Z_1)*100,"%")
print("Testing R2:", r2_score(y_test, Z_2), "RMSE:", np.sqrt(mean_squared_error(y_test, Z_2)), "MAE:", mean_absolute_error(y_test, Z_2), "MAPE:", MAPE(y_test,Z_2)*100,"%")
Z_3 = model.predict(X)

xx = np.zeros(shape=(len(X),2))
xx[:,0]=y
xx[:,1]=Z_3
np.savetxt('XGB.csv', xx, delimiter = ',') 
xx = np.linspace(-0.5, 0.5)
yy = xx
plt.style.use('default')
plt.figure(figsize=(5,4.5),dpi=100)
plt.plot(xx, yy, linewidth=2,c='red')
plt.scatter(y_train, Z_1, marker='o')
plt.scatter(y_test, Z_2, marker='s')

plt.tick_params (axis='both',which='major',labelsize=18)
plt.yticks(fontproperties = 'Times New Roman', size = 18)
plt.xticks(fontproperties = 'Times New Roman', size = 18)

font1 = {'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 20,}
font2 = {'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 15,}
plt.axis('tight')
plt.xlabel('Tested Growth loss', font1)
plt.ylabel('Predicted Growth loss', font1)
plt.yticks([-0.5,0,0.5])
plt.xticks([-0.5,0,0.5])
plt.text(0,0.4,'Humid region', font2)
plt.text(0.1,-0.4,'Training '+"R\u00B2" +'= 0.53', font2)
plt.text(0.1,-0.3,'Testing '+ "R\u00B2"+'= 0.32', font2)
plt.legend(['y = x','Training','Testing'], loc = 'upper left', prop={'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 14,})

plt.savefig('FigS10b.jpg', dpi=600, bbox_inches = 'tight')
plt.show()
X_shap = pd.DataFrame(X, columns = X.columns)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_shap)
print(shap_values.shape)
np.savetxt('shap values.csv', shap_values, delimiter = ',') 
y_base = explainer.expected_value
group_humid['pred'] = model.predict(X)

shap.force_plot(explainer.expected_value, shap_values[2], X.iloc[2], matplotlib=True)
shap.force_plot(explainer.expected_value, shap_values[16], X.iloc[16], matplotlib=True)
shap.force_plot(explainer.expected_value, shap_values, X)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["xtick.labelsize"] = 12
plt.rcParams["ytick.labelsize"] = 12
mean_abs_shap_values = np.mean(np.abs(shap_values), axis=0)
percentages = mean_abs_shap_values / np.sum(mean_abs_shap_values)
data = pd.DataFrame({
    'feature': X_shap.columns,
    'shap_value': mean_abs_shap_values,
    'percentage': percentages
})

data = data.sort_values(by='shap_value', ascending=True)
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 10))
plt.subplots_adjust(wspace=0.001)
ax1.barh(data['feature'], data['shap_value'], color='lightblue')
for i, (feature, shap_value, percentage) in enumerate(data.itertuples(index=False)):
    ax1.text(shap_value + 0.001, i, f"{shap_value:.3f}", va='center', color='blue')
    ax1.text(shap_value + 0.004, i, f"({percentage:.2%})", va='center', color='red')
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_xlabel('Mean(SHAP value)',fontsize=14)
shap.summary_plot(shap_values, X_shap, show=False)
plt.savefig('Fig6b.jpg', dpi=800, bbox_inches = 'tight')
plt.show()